#Домашняя работа № 3. Гибридные рекомендательные системы

##Набор данных MovieLens объемом 100 тыс.

Посмотрим на оригинальный датасет

In [20]:
import pandas as pd
import numpy as np

In [17]:
# загрузим данные
!wget 'https://files.grouplens.org/datasets/movielens/ml-100k.zip'

--2025-02-01 11:45:18--  https://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  8.96MB/s    in 0.5s    

2025-02-01 11:45:19 (8.96 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]



In [22]:
# распакуем
import zipfile

with zipfile.ZipFile('/content/ml-100k.zip', 'r') as zip_ref:
    zip_ref.extractall('unpacked_files')

In [ ]:
# прочитаем описание
"""
ПОДРОБНЫЕ ОПИСАНИЯ ФАЙЛОВ ДАННЫХ
==============================================

Вот краткое описание данных.

ml-data.tar.gz -- Сжатый файл tar.  Чтобы восстановить файлы данных u, выполните следующее:
                gunzip ml-data.tar.gz
                tar xvf ml-data.tar
                mku.sh

u.data - Полный набор u-данных, 100 000 оценок 943 пользователями по 1682 фильмам.
         Каждый пользователь поставил оценку как минимум 20 фильмам. Пользователи и материалы
         нумеруются последовательно, начиная с 1.  Данные получены случайным образом.
         Это разделенный табуляцией список идентификаторов
         пользователей | элементов | оценок | временных меток.
         Временные метки указаны в секундах с 1.01.1970 UTC в системе unix

u.info - Количество пользователей, элементов и оценок в наборе данных u.

u.item - Информация о фильмах, которые будут показаны в этом разделе;
         список фильмов, которые будут показаны в виде вкладок:
         идентификатор фильма | название фильма | дата выхода | дата выхода видео |
         URL на IMDb | неизвестен | Боевик | Приключения | Анимация |
         Детский | Комедийный | Криминальный | Документальный | Драма | фэнтези |
         Фильм-Нуар | Ужасы | Мюзикл | Мистика | Мелодрама | Научная фантастика |
         Триллер | Военный | Вестерн |
         Последние 19 полей обозначают жанры, 1 указывает на фильм относится к этому жанру,
         0 означает, что это не так; фильмы могут быть в сразу в нескольких жанрах.
         Идентификаторы фильмов используются в наборе данных u.data.

u.genre - список жанров.

u.user - Демографическая информация о пользователях; это вкладка разделенный список
         идентификаторов пользователей
         | возраст | пол | род занятий | почтовый индекс.
         Идентификаторы пользователей используются в наборе данных u.data.

u.occupation - Список профессий.

u1.base - Наборы данных u1.base и u1.test через u5.base и u5.test
u1.test представляют собой 80% /20% разбиений данных u на обучающие и тестовые данные.
u2.base Каждый из наборов тестов u1, ..., u5 имеет непересекающиеся наборы тестов;
        это если для
u2.test 5-кратной перекрестной проверки (где вы повторяете свой эксперимент
        u3.base для каждого обучающего и тестового набора и усредняете результаты).
u3.test Эти наборы данных могут быть сгенерированы из u.data с помощью mku.sh.
u4.base
u4.test
u5.base
u5.test

ua.base - Наборы данных ua.base, ua.test, ub.base и ub.test
ua.test разделяют данные u на обучающий набор и тестовый набор с
ub.base ровно по 10 оценок для каждого пользователя в тестовом наборе.  Наборы
ub.test ua.test и ub.test не пересекаются.  Эти наборы данных могут
быть сгенерированы из u.data с помощью mku.sh.

allbut.pl -- Скрипта, который генерирует обучающие и тестовые наборы, где
             все оценки пользователей, кроме n, содержатся в обучающих данных.

mku.sh -- Командный скрипт для генерации всех u наборов данных из u.data.

"""

In [63]:
# прочитаем
u_data = pd.read_csv('/content/unpacked_files/ml-100k/u.data')
u_info = pd.read_csv('/content/unpacked_files/ml-100k/u.info')

# так читать не хочет
# u_item = pd.read_csv('/content/unpacked_files/ml-100k/u.item')
# попробуем по-другому
import csv
csv_file = open('/content/unpacked_files/ml-100k/u.item', newline='', encoding='latin-1')
u_item = pd.read_csv(csv_file, delimiter='|')

u_genre = pd.read_csv('/content/unpacked_files/ml-100k/u.genre')
u_user = pd.read_csv('/content/unpacked_files/ml-100k/u.user')
u_occupation = pd.read_csv('/content/unpacked_files/ml-100k/u.occupation')

In [64]:
# пользователm | элемент | оценка | временная метка
u_data.head()

,196\t242\t3\t881250949
0,186\t302\t3\t891717742
1,22\t377\t1\t878887116
2,244\t51\t2\t880606923
3,166\t346\t1\t886397596
4,298\t474\t4\t884182806


In [65]:
u_info.head()

,943 users
0,1682 items
1,100000 ratings


In [66]:
# идентификатор фильма | название фильма | дата выхода | дата выхода видео |
# URL на IMDb | неизвестен | Боевик | Приключения | Анимация |
# Детский | Комедийный | Криминальный | Документальный | Драма | фэнтези |
# Фильм-Нуар | Ужасы | Мюзикл | Мистика | Мелодрама | Научная фантастика |
# Триллер | Военный | Вестерн |
u_item.head()

,1,Toy Story (1995),01-Jan-1995,Unnamed: 3,http://us.imdb.com/M/title-exact?Toy%20Story%20(1995),0,0.1,0.2,1.1,1.2,...,0.6,0.7,0.8,0.9,0.10,0.11,0.12,0.13,0.14,0.15
0,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,6,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,01-Jan-1995,NaN,http://us.imdb.com/Title?Yao+a+yao+yao+dao+wai...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [67]:
u_genre.head()

,unknown|0
0,Action|1
1,Adventure|2
2,Animation|3
3,Children's|4
4,Comedy|5


In [68]:
# возраст | пол | род занятий | почтовый индекс
u_user.head()

,1|24|M|technician|85711
0,2|53|F|other|94043
1,3|23|M|writer|32067
2,4|24|M|technician|43537
3,5|33|F|other|15213
4,6|42|M|executive|98101


In [69]:
u_occupation.head()

,administrator
0,artist
1,doctor
2,educator
3,engineer
4,entertainment


##LightFM (default)

In [70]:
import numpy as np

In [71]:
!pip install lightfm

In [72]:
from lightfm.datasets import fetch_movielens

movielens = fetch_movielens()

In [73]:
for key, value in movielens.items():
    print(key, type(value), value.shape)

train <class 'scipy.sparse._coo.coo_matrix'> (943, 1682)
test <class 'scipy.sparse._coo.coo_matrix'> (943, 1682)
item_features <class 'scipy.sparse._csr.csr_matrix'> (1682, 1682)
item_feature_labels <class 'numpy.ndarray'> (1682,)
item_labels <class 'numpy.ndarray'> (1682,)


In [74]:
train = movielens['train']
test = movielens['test']

In [16]:
train.toarray()

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]], dtype=int32)

In [76]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

model = LightFM()
model.fit(train, epochs=10)

train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10, train_interactions=train).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))

Precision: train 0.43, test 0.13.


In [77]:
model = LightFM(learning_rate=0.05, loss='warp')

model.fit_partial(train, epochs=10)

train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10, train_interactions=train).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))

Precision: train 0.62, test 0.23.


In [78]:
def sample_recommendation(model, data, user_ids):
    n_users, n_items = data['train'].shape
    for user_id in user_ids:
        known_positives = data['item_labels'][data['train'].tocsr()
                          [user_id].indices]

        scores = model.predict(user_id, np.arange(n_items))

        top_items = data['item_labels'][np.argsort(-scores)]

        print("User %s" % user_id)
        print("     Known positives:")

        for x in known_positives[:3]:
            print("        %s" % x)

        print("     Recommended:")

        for x in top_items[:3]:
            print("        %s" % x)

In [79]:
sample_recommendation(model, movielens, [10, 25, 451])

User 10
     Known positives:
        B
        D
        S
     Recommended:
        R
        S
        R
User 25
     Known positives:
        T
        T
        D
     Recommended:
        S
        F
        C
User 451
     Known positives:
        T
        B
        P
     Recommended:
        R
        S
        C
